## Preprocessing

In [56]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [57]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [58]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [59]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [60]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13', 'T12', 'T2', 'T25','T14','T15','T29','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [61]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C6100        1
C1728        1
C2570        1
C5200        1
C1580        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [62]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C0           3
C2710        3
C1260        3
C3200        2
C1267        2
C1234        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [63]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1000].to_frame()
classifications.columns = ['count']
classifications = classifications.reset_index()
classifications_to_replace = classifications['index']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [64]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(application_df)
print(X_dummies.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [65]:
# Split our preprocessed data into our features and target arrays
y = X_dummies['IS_SUCCESSFUL']
X = X_dummies.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [94]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(25724, 43)

## Compile, Train and Evaluate the Model

In [112]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="tanh", input_dim=43))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 3)                 132       
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 4         
Total params: 148
Trainable params: 148
Non-trainable params: 0
_________________________________________________________________


In [113]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [114]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 1s 974us/step - loss: 0.6231 - accuracy: 0.6759
Epoch 2/100
804/804 [==============================] - 1s 975us/step - loss: 0.5791 - accuracy: 0.7232
Epoch 3/100
804/804 [==============================] - 1s 935us/step - loss: 0.5728 - accuracy: 0.7261
Epoch 4/100
804/804 [==============================] - 1s 999us/step - loss: 0.5706 - accuracy: 0.7288
Epoch 5/100
804/804 [==============================] - 1s 964us/step - loss: 0.5692 - accuracy: 0.7283
Epoch 6/100
804/804 [==============================] - 1s 908us/step - loss: 0.5682 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 1s 917us/step - loss: 0.5677 - accuracy: 0.7298
Epoch 8/100
804/804 [==============================] - 1s 929us/step - loss: 0.5673 - accuracy: 0.7310
Epoch 9/100
804/804 [==============================] - 1s 928us/step - loss: 0.5670 - accuracy: 0.7306
Epoch 10/100
804/804 [==============================] - 1s 927us/step - l

804/804 [==============================] - 1s 980us/step - loss: 0.5627 - accuracy: 0.7310
Epoch 81/100
804/804 [==============================] - 1s 927us/step - loss: 0.5624 - accuracy: 0.7315
Epoch 82/100
804/804 [==============================] - 1s 923us/step - loss: 0.5623 - accuracy: 0.7314
Epoch 83/100
804/804 [==============================] - 1s 936us/step - loss: 0.5624 - accuracy: 0.7317
Epoch 84/100
804/804 [==============================] - 1s 908us/step - loss: 0.5625 - accuracy: 0.7312
Epoch 85/100
804/804 [==============================] - 1s 919us/step - loss: 0.5626 - accuracy: 0.7316
Epoch 86/100
804/804 [==============================] - 1s 912us/step - loss: 0.5623 - accuracy: 0.7321
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5624 - accuracy: 0.7316
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5624 - accuracy: 0.7316
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.562

In [115]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 0s 859us/step - loss: 0.5667 - accuracy: 0.7306
Loss: 0.5666944980621338, Accuracy: 0.7306122183799744


In [116]:
# Export our model to HDF5 file
nn_model.save_weights("AlphabetSoupCharity.h5");